# 🧪 ZeroLang Environment Test

**Run this first to verify everything works before the full pipeline.**

Takes ~2 minutes. Tests:
1. LLVM/Clang WASM compilation
2. wasm-tools installation
3. Repository cloning
4. Data pipeline
5. GPU availability

In [ ]:
import subprocess
import sys

results = {}

def test(name, func):
    """Run a test and record result."""
    print(f"\n[{name}] Testing...")
    try:
        result = func()
        results[name] = "✓ PASS"
        print(f"[{name}] ✓ PASS")
        return result
    except Exception as e:
        results[name] = f"✗ FAIL: {e}"
        print(f"[{name}] ✗ FAIL: {e}")
        return None

In [ ]:
# Test 1: Install LLVM
def test_llvm_install():
    result = subprocess.run(
        "apt-get update -qq && apt-get install -qq -y llvm lld clang",
        shell=True, capture_output=True, text=True
    )
    # Verify clang exists
    result = subprocess.run(["which", "clang"], capture_output=True, text=True)
    if result.returncode != 0:
        raise Exception("clang not found after install")
    return result.stdout.strip()

test("LLVM Install", test_llvm_install)

In [ ]:
# Test 2: WASM Compilation
def test_wasm_compile():
    # Write test file
    with open('/tmp/test_func.c', 'w') as f:
        f.write('int add(int a, int b) { return a + b; }')
    
    # Compile to WASM
    result = subprocess.run(
        [
            "clang", "--target=wasm32", "-O2", "-nostdlib",
            "-fuse-ld=lld", "-Wl,--no-entry", "-Wl,--export-all",
            "-o", "/tmp/test_func.wasm", "/tmp/test_func.c"
        ],
        capture_output=True, text=True
    )
    if result.returncode != 0:
        raise Exception(f"Compilation failed: {result.stderr}")
    
    import os
    if not os.path.exists('/tmp/test_func.wasm'):
        raise Exception("WASM file not created")
    
    return f"Created {os.path.getsize('/tmp/test_func.wasm')} bytes"

test("WASM Compile", test_wasm_compile)

In [ ]:
# Test 3: Install wasm-tools
def test_wasm_tools():
    # Download and install
    cmds = [
        "cd /tmp && curl -sLO https://github.com/bytecodealliance/wasm-tools/releases/download/v1.244.0/wasm-tools-1.244.0-x86_64-linux.tar.gz",
        "cd /tmp && tar -xzf wasm-tools-1.244.0-x86_64-linux.tar.gz",
        "cp /tmp/wasm-tools-1.244.0-x86_64-linux/wasm-tools /usr/local/bin/",
        "chmod +x /usr/local/bin/wasm-tools"
    ]
    for cmd in cmds:
        result = subprocess.run(cmd, shell=True, capture_output=True, text=True)
        if result.returncode != 0:
            raise Exception(f"Failed: {cmd}")
    
    # Verify version
    result = subprocess.run(["wasm-tools", "--version"], capture_output=True, text=True)
    if result.returncode != 0:
        raise Exception("wasm-tools not working")
    return result.stdout.strip()

test("wasm-tools", test_wasm_tools)

In [ ]:
# Test 4: WASM to WAT conversion
def test_wat_conversion():
    result = subprocess.run(
        ["wasm-tools", "print", "/tmp/test_func.wasm"],
        capture_output=True, text=True
    )
    if result.returncode != 0:
        raise Exception(f"Conversion failed: {result.stderr}")
    
    wat = result.stdout
    if "(module" not in wat or "func" not in wat:
        raise Exception("Invalid WAT output")
    
    print("WAT output preview:")
    print(wat[:500])
    return f"{len(wat)} chars"

test("WAT Conversion", test_wat_conversion)

In [ ]:
# Test 5: Git clone
def test_git_clone():
    import shutil
    import os
    
    if os.path.exists('/tmp/test_repo'):
        shutil.rmtree('/tmp/test_repo')
    
    result = subprocess.run(
        ["git", "clone", "--depth=1", "https://github.com/983/SHA-256", "/tmp/test_repo"],
        capture_output=True, text=True
    )
    if result.returncode != 0:
        raise Exception(f"Clone failed: {result.stderr}")
    
    # Check if C files exist
    c_files = list(__import__('pathlib').Path('/tmp/test_repo').rglob('*.c'))
    return f"Found {len(c_files)} C files"

test("Git Clone", test_git_clone)

In [ ]:
# Test 6: GPU
def test_gpu():
    import torch
    if not torch.cuda.is_available():
        raise Exception("No GPU available")
    
    gpu_name = torch.cuda.get_device_name(0)
    gpu_mem = torch.cuda.get_device_properties(0).total_memory / 1e9
    return f"{gpu_name} ({gpu_mem:.1f} GB)"

test("GPU", test_gpu)

In [ ]:
# Test 7: Python dependencies
def test_deps():
    result = subprocess.run(
        [sys.executable, "-m", "pip", "install", "-q", 
         "transformers", "datasets", "peft", "accelerate", "bitsandbytes"],
        capture_output=True, text=True
    )
    
    # Verify imports
    import transformers
    import datasets
    import peft
    return f"transformers {transformers.__version__}"

test("Python Deps", test_deps)

In [ ]:
# Test 8: Clone ZeroLang repo and test pipeline
def test_pipeline():
    import shutil
    import os
    
    if os.path.exists('/content/zerolang_test'):
        shutil.rmtree('/content/zerolang_test')
    
    # Clone
    result = subprocess.run(
        ["git", "clone", "--depth=1", "https://github.com/project-zero-git/zerolang.git", "/content/zerolang_test"],
        capture_output=True, text=True
    )
    if result.returncode != 0:
        raise Exception(f"Clone failed: {result.stderr}")
    
    # Update clang path
    gen_path = '/content/zerolang_test/pipeline/generator.py'
    with open(gen_path) as f:
        content = f.read()
    content = content.replace('/opt/homebrew/opt/llvm/bin/clang', 'clang')
    with open(gen_path, 'w') as f:
        f.write(content)
    
    # Run with a small test
    result = subprocess.run(
        [sys.executable, "pipeline/generator.py", 
         "-r", "https://github.com/983/SHA-256",
         "-o", "/tmp/test_output.jsonl"],
        capture_output=True, text=True,
        cwd="/content/zerolang_test"
    )
    
    # Check output
    import os
    if os.path.exists('/tmp/test_output.jsonl'):
        with open('/tmp/test_output.jsonl') as f:
            lines = f.readlines()
        return f"Generated {len(lines)} pairs"
    else:
        print("stdout:", result.stdout[-500:])
        print("stderr:", result.stderr[-500:])
        raise Exception("No output file created")

test("Full Pipeline", test_pipeline)

In [ ]:
# Summary
print("\n" + "="*60)
print("TEST SUMMARY")
print("="*60)

all_passed = True
for name, result in results.items():
    status = "✓" if "PASS" in result else "✗"
    print(f"  {status} {name}: {result}")
    if "FAIL" in result:
        all_passed = False

print("\n" + "="*60)
if all_passed:
    print("🎉 ALL TESTS PASSED!")
    print("You can now run the full pipeline notebook.")
else:
    print("⚠️ SOME TESTS FAILED")
    print("Fix the issues before running the full pipeline.")
print("="*60)